# Projeto 2 - Ciência dos Dados

João Otávio Gentil Palma

# Prevendo o rendimento médio de fazendas pelo Brasil
___
## Introdução

### O Objetivo

O objetivo do projeto 2 de Ciência dos Dados é **prever**, por meio de algumas variáveis $features$, **uma variável $target$**. Em termos mais comuns, isso significa dizer que, com base em variáveis que explicam ou afetam um fenômeno, ou seja, que têm correlação com ele, vamos prever esse fenômeno, seja ele um número (variável quantitativa) ou classificação (variável qualitativa).

### Rendimento médio de uma plantação

No caso do tema específico do projeto, vamos **prever o rendimento médio de uma safra (colheita)** com base em vários fatores que se correlacionam com o preço, como a localização, produto, área plantada, área colhida, quantidade produzida, valor_producao, entre outros.

In [1]:
import pandas as pd
import seaborn as sns

# Filtros 
Em seguida filtraremos o dataframe utilizado para trabalharmos com apenas as 'features' desejadas.

In [2]:
# Explorando df de lavouras temporarias

dados_temporario = pd.read_csv("dados_lavouras_temporario_na.csv")
dados_temporario = dados_temporario.drop(columns=['Unnamed: 0', 'id_municipio'])
dados_temporario = dados_temporario.dropna()
dados_temporario['Tipo'] = 'Temporário'
dados_temporario

# Explorando df de lavouras permanentes

dados_permanente = pd.read_csv("dados_lavouras_permanente_na.csv")
dados_permanente = dados_permanente.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'id_municipio'])
dados_permanente['Tipo'] = 'Permanente'
dados_permanente


# Concatenando ambas as planilhas
dados_lavouras = pd.concat([dados_permanente, dados_temporario], axis=0)
dados_lavouras = dados_lavouras.dropna()
dados_lavouras

,ano,sigla_uf,produto,area_plantada,area_colhida,quantidade_produzida,rendimento_medio,valor_producao,prop_area_plantada,prop_area_colhida,prop_valor_producao,Tipo
0,2007,BA,Manga,85.0,85.0,340.0,4000.0,88.0,11.02,11.02,8.55,Permanente
1,2007,BA,Sisal ou agave (fibra),60.0,60.0,60.0,1000.0,54.0,0.75,0.76,0.28,Permanente
2,2007,BA,Coco-da-baía,19.0,15.0,75.0,5000.0,30.0,18.63,15.31,10.49,Permanente
3,2007,BA,Maracujá,3.0,3.0,45.0,15000.0,9.0,0.51,0.51,0.16,Permanente
4,2007,BA,Limão,5.0,5.0,78.0,15600.0,32.0,0.11,0.12,0.23,Permanente
...,...,...,...,...,...,...,...,...,...,...,...,...
8444613,2018,MG,Tomate,5.0,5.0,290.0,58000.0,411.0,0.03,0.03,0.50,Temporário
8444618,2018,MG,Tomate,1.0,1.0,35.0,35000.0,47.0,0.11,0.11,0.77,Temporário
8444620,2018,MG,Feijão (em grão),30.0,30.0,24.0,800.0,72.0,10.27,10.27,6.99,Temporário
8444621,2018,MG,Sorgo (em grão),3.0,3.0,9.0,3000.0,3.0,1.03,1.03,0.29,Temporário


In [3]:
#dados_lavouras['ano'].value_counts()

In [5]:
# Removendo as linhas dos anos 1994 ou antes, pois a moeda era diferente

dados_lavouras = dados_lavouras.drop(dados_lavouras[dados_lavouras.ano.isin([1991, 1992, 1989, 1993])].index).reset_index(drop=True)

Até o momento, foi feito um filtro nos dados da planilha para que ela fique masi prática e fácil de trabalhar

Tirei anos anteriores a 1994, pois a moeda que circulava era diferente

In [7]:
dados_lavouras

,ano,sigla_uf,produto,area_plantada,area_colhida,quantidade_produzida,rendimento_medio,valor_producao,prop_area_plantada,prop_area_colhida,prop_valor_producao,Tipo
0,2007,BA,Manga,85.0,85.0,340.0,4000.0,88.0,11.02,11.02,8.55,Permanente
1,2007,BA,Sisal ou agave (fibra),60.0,60.0,60.0,1000.0,54.0,0.75,0.76,0.28,Permanente
2,2007,BA,Coco-da-baía,19.0,15.0,75.0,5000.0,30.0,18.63,15.31,10.49,Permanente
3,2007,BA,Maracujá,3.0,3.0,45.0,15000.0,9.0,0.51,0.51,0.16,Permanente
4,2007,BA,Limão,5.0,5.0,78.0,15600.0,32.0,0.11,0.12,0.23,Permanente
...,...,...,...,...,...,...,...,...,...,...,...,...
1650072,2018,MG,Tomate,5.0,5.0,290.0,58000.0,411.0,0.03,0.03,0.50,Temporário
1650073,2018,MG,Tomate,1.0,1.0,35.0,35000.0,47.0,0.11,0.11,0.77,Temporário
1650074,2018,MG,Feijão (em grão),30.0,30.0,24.0,800.0,72.0,10.27,10.27,6.99,Temporário
1650075,2018,MG,Sorgo (em grão),3.0,3.0,9.0,3000.0,3.0,1.03,1.03,0.29,Temporário


In [ ]:
# Analisando a relação entre as variáveis

# sns.pairplot(dados_lavouras, height=1.5);
dados_lavouras.corr()

In [12]:
norte = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
nordeste = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
centro_oeste = ['MT', 'MS', 'GO']
sudeste = ['SP', 'RJ', 'ES', 'MG']
sul = ['PR', 'RS', 'SC']
def func_regiao(sigla):
    if sigla in norte:
        return 'Norte'
    elif sigla in nordeste:
        return 'Nordeste'
    elif sigla in centro_oeste:
        return 'Centro-oeste'
    elif sigla in sudeste:
        return 'Sudeste'
    else:
        return 'Sul'
        
dados_lavouras['regiao'] = dados_lavouras['sigla_uf'].apply(func_regiao)

In [18]:
dados_lavouras = dados_lavouras[['ano', 'sigla_uf','regiao', 'produto', 'area_plantada', 'area_colhida',
       'quantidade_produzida', 'rendimento_medio', 'valor_producao',
       'prop_area_plantada', 'prop_area_colhida', 'prop_valor_producao',
       'Tipo']]

dados_lavouras.loc[dados_lavouras['regiao'] == 'Sudeste'].corr()

,ano,area_plantada,area_colhida,quantidade_produzida,rendimento_medio,valor_producao,prop_area_plantada,prop_area_colhida,prop_valor_producao
ano,1.000000,0.043932,0.043413,0.035176,-0.110303,0.151149,0.071284,0.071434,0.068816
area_plantada,0.043932,1.000000,0.999181,0.736261,0.105996,0.768746,0.354975,0.354728,0.336166
area_colhida,0.043413,0.999181,1.000000,0.736346,0.106227,0.769519,0.354365,0.354633,0.336371
quantidade_produzida,0.035176,0.736261,0.736346,1.000000,0.251422,0.610241,0.251133,0.251225,0.265859
rendimento_medio,-0.110303,0.105996,0.106227,0.251422,1.000000,0.107942,-0.003588,-0.002434,0.112955
valor_producao,0.151149,0.768746,0.769519,0.610241,0.107942,1.000000,0.339656,0.339909,0.362377
prop_area_plantada,0.071284,0.354975,0.354365,0.251133,-0.003588,0.339656,1.000000,0.998515,0.896487
prop_area_colhida,0.071434,0.354728,0.354633,0.251225,-0.002434,0.339909,0.998515,1.000000,0.899215
prop_valor_producao,0.068816,0.336166,0.336371,0.265859,0.112955,0.362377,0.896487,0.899215,1.000000
